# Reading in MCTS (Monte Carlo Tree Search) data
### The goal is to train two RNNs.
 - Predict Probability of each of the moves being successful
 - Evaluate the value of the current board position

In [3]:
# uninstall numpy and then reinstall/update tensorflow
import pandas as pd
import numpy as np
print(np.__version__) # must not be the most recent. 1.26.4 works.
import tensorflow as tf
print(tf.__version__)
  
from board import Board


1.26.4
2.17.0


In [5]:
import pandas as pd 
print(pd.__version__) # should work with 2.2.2
from board import Board

filename = 'Data/playing_data.pkl'


df = pd.read_pickle('Data/playing_data.pkl')
cols = ['Board', 'Moves','Move_Probs','Reward', 'Weights']

print(df.head)
print(df.ndim, ' dimensions')
print(df.shape, ' shape')
print(df.size, ' size')

2.2.2
<bound method NDFrame.head of     level_0  index                                              Board  \
0       0.0    0.0  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
1       1.0    1.0  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
2       2.0    2.0  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
3       3.0    3.0  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
4       4.0    4.0  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [...   
..      ...    ...                                                ...   
49      NaN    NaN  [[0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0], [...   
50      NaN    NaN  [[0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0], [...   
51      NaN    NaN  [[0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0], [...   
52      NaN    NaN  [[0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0], [...   
53      NaN    NaN  [[0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0], [...   

                                                Moves  \
0   [[4, 7, 20, 7, 0, 3], [4, 

In [20]:
# checking out to make the data is stored correctly
  
row = 14
print(df.iloc[row])
print(df.iloc[row]['inv_left'])
x = df.iloc[row]['Board']
board = Board(14)
board.show_dots = False
board.board = x
print(board)
print(df.iloc[row]['Move_Probs'])
print(df.iloc[row]['Moves'])

print(df.iloc[row]['Board'])


Board         [[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [...
Moves         [[3, 3, 0, 0, 7, 1], [5, 8, 0, 0, 14, 2], [3, ...
Move_Probs    [0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.0...
Reward                                                       -1
Weights       [3.107913851073312, 43.518217740455015, 49.539...
Num_Sims                                                     50
mc_type                                       random_monteCarlo
inv_left                       [4, 6, 7, 8, 11, 14, 16, 18, 19]
Name: 14, dtype: object
[4, 6, 7, 8, 11, 14, 16, 18, 19]
   A B C D E F G H I J K L M N
 1 □ □ □ ▣ □ □ ▣ □ □ □ □ □ □ □ 
 2 □ □ ▣ ▣ □ ▣ ▣ □ □ □ ▣ ▣ ▣ ▣ 
 3 □ □ ▣ □ ▣ ▣ □ ▣ ▣ ▣ □ □ □ □ 
 4 □ ▣ □ □ □ □ ▣ □ □ ▣ □ □ □ □ 
 5 ▣ ▣ ▣ □ ▣ ▣ ▣ □ □ ▣ □ □ □ □ 
 6 □ ▣ □ □ □ □ ▣ □ □ ▣ ▣ ▣ □ □ 
 7 □ ▣ □ □ ▣ ▣ □ ▣ □ □ ▣ □ □ □ 
 8 □ ▣ □ ▣ ▣ □ ▣ ▣ □ □ ▣ □ □ □ 
 9 □ ▣ ▣ □ ▣ □ □ □ □ ▣ □ ▣ □ □ 
10 □ □ □ □ □ ▣ ▣ □ ▣ ▣ □ ▣ □ □ 
11 □ □ □ □ □ ▣ □ ▣ ▣ □ □ ▣ □ □ 
12 □ ▣ ▣ ▣ ▣ □ □ □ □ □ □ ▣ □ □ 
13 □ ▣ □ □ □ □ □

# Working on the DCNNs
### References:
 - https://medium.com/aiguys/deep-convolutional-neural-networks-dcnns-explained-in-layman-terms-b990b2818061
  - https://nikcheerla.github.io/deeplearningschool/2018/01/01/AlphaZero-Explained/


In [25]:
# Data preprocessing
import math

# df = df.sample(frac = 1)
# training_ratio = 0.8
# split_index = math.floor(training_ratio*(df.shape[0]))
# print("Amount of training batches: ", split_index)
# train_df = df.iloc[:split_index]
# test_df = df.iloc[split_index:]

# train_boards = np.array(tuple(train_df['Board'].values))
# train_rewards = np.array(tuple(train_df['Reward'].values))

# test_boards = np.array(tuple(test_df['Board'].values))
# test_rewards = np.array(tuple(test_df['Reward'].values))

# print("Shape of training boards: ", np.shape(train_boards))

In [75]:
def preprocess_data(games, train_test_ratio=.8):
    # Convert game records into input-output pairs
    games = games.sample(frac = 1)
    split_index = math.floor(train_test_ratio*(games.shape[0]))
    sameation_num = 10
    print("Amount of training batches: ", split_index)
    train_df = games.iloc[:split_index-sameation_num]
    test_df = games.iloc[split_index:]
    sameation_df = games.iloc[split_index-sameation_num:split_index]

    train_boards = np.array(tuple(train_df['Board'].values))
    train_rewards = np.array(tuple(train_df['Reward'].values))

    test_boards = np.array(tuple(test_df['Board'].values))
    test_rewards = np.array(tuple(test_df['Reward'].values))
    
    x_val = np.array(tuple(sameation_df['Board'].values))
    y_val = np.array(tuple(sameation_df['Reward'].values))
    
    train_boards = np.reshape(train_boards, (train_boards.shape[0], train_boards.shape[1], train_boards.shape[2], 1))

    
    processed_data = {
        "train boards": train_boards,
        "train rewards": train_rewards,
        "test boards": test_boards,
        "test rewards": test_rewards,
        "x val": x_val,
        "y val": y_val
    }
    
    # Normalize and format the data for training
    return processed_data

In [158]:
# Defining the evaluation snd predictive networks
def create_policy_network(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(input_shape[0] * input_shape[1], activation='softmax')
    ])
    return model

def create_value_network(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        tf.keras.layers.Conv2D(64, kernel_size=3, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(1, activation='tanh')
    ])
    return model


# def create_value_network1(input_shape):
#     model = tf.keras.Sequential([

#     tf.keras.layers.Conv2D(filters = 32, input_shape = input_shape,  
#                 kernel_size = (7, 7),  
#                 padding = 'same'),
#     tf.keras.layers.Activation('relu'),
#     # Max-Pooling  
#     # tf.keras.layers.MaxPooling2D(pool_size = (2, 2), 
#     #             strides = (2, 2), padding = 'same'),
#     # # Batch Normalisation 
#     tf.keras.layers.BatchNormalization(), 
    
#     # 2nd Convolutional Layer 
#     tf.keras.layers.Conv2D(filters = 30, kernel_size = (11, 11),  
#                 strides = (1, 1), padding = 'same'), 
#     tf.keras.layers.Activation('relu'), 
#     # Max-Pooling 
#     tf.keras.layers.MaxPooling2D(pool_size = (2, 2),  
#                 padding = 'same'),
#     # Batch Normalisation 
#     tf.keras.layers.BatchNormalization(),
    
#     # 3rd Convolutional Layer 
#     tf.keras.layers.Conv2D(filters = 384, kernel_size = (3, 3),  
#                 strides = (1, 1), padding = 'same'),
#     tf.keras.layers.Activation('relu'),
#     # Batch Normalisation 
#     tf.keras.layers.BatchNormalization(),
    
#     # 4th Convolutional Layer 
#     tf.keras.layers.Conv2D(filters = 384, kernel_size = (3, 3),  
#                 strides = (1, 1), padding = 'same'),
#     tf.keras.layers.Activation('relu'),
#     # Batch Normalisation 
#     tf.keras.layers.BatchNormalization(),
    
#     # 5th Convolutional Layer 
#     tf.keras.layers.Conv2D(filters = 256, kernel_size = (3, 3),  
#                 strides = (1, 1), padding = 'same'),
#     tf.keras.layers.Activation('relu'),
#     # Max-Pooling 
#     tf.keras.layers.MaxPooling2D(pool_size = (2, 2), strides = (2, 2),  
#                 padding = 'same'),
#     # Batch Normalisation 
#     tf.keras.layers.BatchNormalization(),
    
#     # Flattening 
#     tf.keras.layers.Flatten(),
    
#     # 1st Dense Layer 
#     tf.keras.layers.Dense(4096, input_shape = (14*14*1, )),
#     tf.keras.layers.Activation('relu'),
#     # Add Dropout to prevent overfitting 
#     tf.keras.layers.Dropout(0.4),
#     # Batch Normalisation 
#     tf.keras.layers.BatchNormalization(),
    
#     # 2nd Dense Layer 
#     tf.keras.layers.Dense(4096),
#     tf.keras.layers.Activation('relu'),
#     # Add Dropout 
#     tf.keras.layers.Dropout(0.4),
#     # Batch Normalisation 
#     tf.keras.layers.BatchNormalization(),
    
#     tf.keras.layers.Dense(1, activation='tanh')

#     ])
#     return model

In [159]:
# Code snippet for training the neural networks
def train_networks(policy_network, value_network, data, epochs=10, batch_size=32):
    # Define loss functions and optimizer
    policy_loss = tf.keras.losses.CategoricalCrossentropy()
    value_loss = tf.keras.losses.MeanSquaredError()
    optimizer = tf.keras.optimizers.Adam()
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]

    # Compile the models
    policy_network.compile(optimizer=optimizer, loss=policy_loss, metrics=metrics)
    value_network.compile(optimizer=optimizer, loss=value_loss, metrics=metrics)

    # Train the models
    # policy_network.fit(processed_data['input'], processed_data['policy_target'], epochs=epochs, batch_size=batch_size, sameation_data=(x_val, y_val))
    value_network.fit(data['train boards'], data['train rewards'], epochs=epochs, batch_size=batch_size, sameation_data=(data['x val'], data['y val']))

In [160]:
board_size = 14
print(np.shape(df))
data = preprocess_data(df, train_test_ratio=.8)
input_shape = (board_size, board_size, 1)

policy_network = create_policy_network(input_shape)
value_network = create_value_network(input_shape)

train_networks(policy_network, value_network, data, epochs=100, batch_size=32)

(44, 8)
Amount of training batches:  35
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\IPython\core\interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\jjfri\AppData\Local\Temp\ipykernel_32776\4204928663.py", line 9, in <module>
    train_networks(policy_network, value_network, data, epochs=100, batch_size=32)
  File "C:\Users\jjfri\AppData\Local\Temp\ipykernel_32776\1916808393.py", line 15, in train_networks
    value_network.fit(data['train boards'], data['train rewards'], epochs=epochs, batch_size=batch_size, sameation_data=(data['x val'], data['y val']))
  File "C:\Users\jjfri\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\User

In [161]:
to_pred = np.array((tuple(df.iloc[14]['Board'])))
to_pred = to_pred[np.newaxis, :, :, np.newaxis]
print(np.shape(to_pred))
value_network.predict(to_pred)

(1, 14, 14, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


array([[0.13369839]], dtype=float32)

In [162]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = value_network.evaluate(data['test boards'], data['test rewards'], batch_size=12)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
for i in range(8):
    # print("Generate predictions for 3 samples")
    to_pred = np.array(tuple(data['test boards'][i]))
    to_pred = to_pred[np.newaxis, :, :, np.newaxis]

    print(np.shape(to_pred))

    predictions = value_network.predict(to_pred)
    print("predictions shape:", predictions.shape)
    print("prediction:", predictions, "correct reward: ", -data['test rewards'][i])

Evaluate on test data
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step - loss: 1.0202 - sparse_categorical_accuracy: 0.0000e+00
test loss, test acc: [1.020215392112732, 0.0]
(1, 14, 14, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
predictions shape: (1, 1)
prediction: [[0.11991046]] correct reward:  -1
(1, 14, 14, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
predictions shape: (1, 1)
prediction: [[0.0925564]] correct reward:  1
(1, 14, 14, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
predictions shape: (1, 1)
prediction: [[0.05946523]] correct reward:  -1
(1, 14, 14, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
predictions shape: (1, 1)
prediction: [[0.12101801]] correct reward:  -1
(1, 14, 14, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
predictions shape: (1, 1)
prediction: [[0.10638861]] correct reward:  -1
(1, 14, 14, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
predictions shape: (1, 1)
prediction: [[0.11931356]] correct reward:  1
(1, 14, 14, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
predictions shape: (1, 1)
pred